# Package & Datasets
Disini import package dan data sekaligus regex untuk membersihkan punctuation(tanda baca)

In [49]:
import pandas as pd
import re
import ipywidgets as widgets
from IPython.display import display

In [62]:
url = 'https://raw.githubusercontent.com/dzaky207/Tools-NIK/main/kecamatan.csv'

In [63]:
df = pd.read_csv(url)

In [64]:
df

,id,name
0,11.01.01,Bakongan
1,11.01.02,Kluet Utara
2,11.01.03,Kluet Selatan
3,11.01.04,Labuhan Haji
4,11.01.05,Meukek
...,...,...
7272,96.71.06,Sorong Manoi
7273,96.71.07,Sorong Kota
7274,96.71.08,Klaurung
7275,96.71.09,Malaimsimsa


In [65]:
def hapus_tanda_baca(teks):
    return re.sub(r'[^\w\s]', '', teks)

# Terapkan fungsi ke kolom 'id'
df['id'] = df['id'].apply(hapus_tanda_baca)

# Encrypt to NIK
Mengubah data wilayah dan tanggal lahir menjadi kode yang memenuhi format NIK

In [69]:
# Membuat dropdown untuk memilih name
name_dropdown = widgets.Combobox(
    options=df['name'].tolist(),
    description='Name:',
    ensure_option=True,
    continuous_update=False
)

# Dropdown pertama berisi angka dari 01 sampai 31
day_dropdown = widgets.Dropdown(
    options=[f'{i:02d}' for i in range(1, 32)],
    description='Hari:',
    disabled=False,
)

# Dropdown kedua berisi angka dari 01 sampai 12
month_dropdown = widgets.Dropdown(
    options=[f'{i:02d}' for i in range(1, 13)],
    description='Bulan:',
    disabled=False,
)

# Dropdown ketiga berisi angka dari 00 sampai 99
year_dropdown = widgets.Dropdown(
    options=[f'{i:02d}' for i in range(0, 100)],
    description='Tahun(2 digit terakhir):',
)

# Dropdown keempat berisi pilihan "Pria" dan "Wanita"
gender_dropdown = widgets.Dropdown(
    options=['Pria', 'Wanita'],
    description='Jenis Kelamin:',
    disabled=False,
)

# Input nomer urut setelah dropdown gender
id_input = widgets.Text(
    placeholder='Masukkan nomer urut',
    description='Nomer Urut:',
    disabled=False
)

# Tombol untuk mencetak pilihan user
button = widgets.Button(description="Submit")

# Label untuk menampilkan hasil
output_label = widgets.Label()

# Fungsi untuk menangani klik tombol
def on_button_click(b):
    name = name_dropdown.value
    day = day_dropdown.value
    month = month_dropdown.value
    year = year_dropdown.value
    gender = gender_dropdown.value
    id_number = id_input.value.strip()  # Mengambil nilai nomer urut dan menghapus spasi di awal dan akhir

    # Menyusun nomer urut menjadi 4 digit dengan menambahkan angka 0 di depan jika kurang dari 4 digit
    id_number = id_number.zfill(4)

    if gender == 'Wanita':
        day = str(int(day) + 40).zfill(2)  # Mengubah day menjadi integer, menambah 40, dan mengubah kembali ke string dengan 2 digit

    if name in df['name'].values:
        selected_id = df[df['name'] == name]['id'].values[0]
        result_string = f"NIK : {selected_id}{day}{month}{year}{id_number}"
    else:
        result_string = "Name not found."

    output_label.value = result_string

# Menghubungkan tombol dengan fungsi
button.on_click(on_button_click)

# Menampilkan dropdown, input nomer urut, tombol, dan label hasil
display(name_dropdown, day_dropdown, month_dropdown, year_dropdown, gender_dropdown, id_input, button, output_label)


Combobox(value='', continuous_update=False, description='Name:', ensure_option=True, options=('Bakongan', 'Klu…

Dropdown(description='Hari:', options=('01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12',…

Dropdown(description='Bulan:', options=('01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'…

Dropdown(description='Tahun(2 digit terakhir):', options=('00', '01', '02', '03', '04', '05', '06', '07', '08'…

Dropdown(description='Jenis Kelamin:', options=('Pria', 'Wanita'), value='Pria')

Text(value='', description='Nomer Urut:', placeholder='Masukkan nomer urut')

Button(description='Submit', style=ButtonStyle())

Label(value='')

# Decrypt from NIK
Menterjemahkan NIK menjadi data berupa wilayah(kecamatan) dan tanggal lahir.

In [71]:
# Masukkan NIK
NIK = input("Masukkan NIK: ")

# Pengecekan panjang NIK
if len(NIK) != 16:
    print("NIK tidak valid. Harus terdiri dari 16 digit.")
else:
    # Memecah NIK menjadi dua bagian
    kw = NIK[:6]
    bagian2 = NIK[6:12]
    bagian3 = NIK[12:]
    # Bagian pertama: Mencocokkan angka dengan kolom id
    if kw in df['id'].values:
        wilayah = df[df['id'] == kw]['name'].values[0]
    else:
        wilayah = "ID tidak ditemukan"

    # Bagian kedua: Memecah menjadi 3 bagian masing-masing 2 angka
    hari_lahir = int(bagian2[:2])
    bulan_lahir = int(bagian2[2:4])
    tahun_lahir = int(bagian2[4:])

    # Klasifikasi jenis kelamin berdasarkan hari lahir
    if hari_lahir > 41:
        jenis_kelamin = "Wanita"
    else:
        jenis_kelamin = "Pria"

    # Cocokkan bulan lahir
    bulan_lahir_dict = {
        1: "Januari", 2: "Februari", 3: "Maret", 4: "April",
        5: "Mei", 6: "Juni", 7: "Juli", 8: "Agustus",
        9: "September", 10: "Oktober", 11: "November", 12: "Desember"
    }
    bulan_lahir_str = bulan_lahir_dict.get(bulan_lahir, "Bulan tidak valid")

    # Tambahkan tahun lahir
    if 25 <= tahun_lahir <= 99:
        tahun_lahir_str = '19' + str(tahun_lahir)
    elif 0 <= tahun_lahir <= 24:
        tahun_lahir_str = '200' + str(tahun_lahir)
    else:
        tahun_lahir_str = "Tahun tidak valid"

    urutan = str(int(bagian3))
    # Output
    print('Wilayah:', wilayah)
    print('Jenis Kelamin:', jenis_kelamin)
    print('Tanggal Lahir(Hari):', hari_lahir)
    print('Bulan Lahir:', bulan_lahir_str)
    print('Tahun Lahir:', tahun_lahir_str)
    print('Nomor Urut:', urutan)

Masukkan NIK: 3309112106610003
Wilayah: Ngemplak
Jenis Kelamin: Pria
Tanggal Lahir(Hari): 21
Bulan Lahir: Juni
Tahun Lahir: 1961
Nomor Urut: 3
